In [1]:
from olinda.distillation import distill

2024-05-16 14:40:53.276894: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-16 14:40:53.276932: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-16 14:40:53.277528: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-16 14:40:53.281158: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-16 14:40:53.968656: W tensorflow/compiler/tf2

Using TensorFlow backend


## A model to distill

You need a trained model for the distillation process. Here we are creating a simple pytorch model(untrained) for demo. A tensorflow model is also supported.

In [2]:
from typing import Any

import pytorch_lightning as pl
from torch import nn
from torch.nn import functional as F


class DemoModel(pl.LightningModule):
    """OlindaNet Zero: Slim(relatively) distillation network."""

    def __init__(
        self: "DemoModel"
    ) -> None:
        """Init."""
        super().__init__()
        self.conv1 = nn.Conv2d(32, 32, kernel_size=3, padding=0, stride=3)
        self.fc1 = nn.Linear(100, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 1)

        

    def forward(self: "DemoModel", x: Any) -> Any:
        """Forward function.

        Args:
            x (Any): model input

        Returns:
            Any: model output
        """
        x = self.conv1(x)
        x = F.relu(x)

        # flattening while keeping the batch axis
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x
###############

model = DemoModel()

## Distillation

Quickly test the distillation pipeline for your model with a small reference smiles dataset

In [3]:
#featurizer, student_model = distill(model, test=True, clean=True)
featurizer, student_model = distill(model, test=True, clean=True, small_data=True) #An even smaller test set

Creating model output: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3125/3125 [01:57<00:00, 26.54it/s]
2024-05-16 14:46:14.801850: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-05-16 14:46:14.801877: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: jason-Legion-5-15ACH6
2024-05-16 14:46:14.801884: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: jason-Legion-5-15ACH6
2024-05-16 14:46:14.801920: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 550.54.15
2024-05-16 14:46:14.801937: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 550.54.15
2024-05-16 14:46:14.801942: I external/local_x

Reloading Tuner from ./trials/tuner0.json
Epoch 1/3
3125/3125 [==============================] - 235s 75ms/step - loss: 0.1133 - val_loss: 1.5815e-04
Epoch 2/3
3125/3125 [==============================] - 226s 72ms/step - loss: 1.4971e-04 - val_loss: 1.3651e-04
Epoch 3/3
3125/3125 [==============================] - 225s 72ms/step - loss: 1.3610e-04 - val_loss: 1.3184e-04
Best epoch: 3
Epoch 1/3
3125/3125 [==============================] - 224s 72ms/step - loss: 0.2662 - val_loss: 1.5401e-04
Epoch 2/3
3125/3125 [==============================] - 224s 72ms/step - loss: 1.4310e-04 - val_loss: 1.3508e-04
Epoch 3/3
3125/3125 [==============================] - 224s 72ms/step - loss: 1.3538e-04 - val_loss: 1.3187e-04


2024-05-16 15:09:00.650862: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-05-16 15:09:00.650963: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-05-16 15:09:00.697568: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-05-16 15:09:00.697666: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


In [4]:
smiles_list = featurizer.featurize(["CCCOC"])
student_model(smiles_list)

[-0.020981745794415474]